In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from aging.plotting import (
    figure,
    format_plots,
    add_identity,
    legend,
    PLOT_CONFIG,
    save_factory,
    COLORMAPS,
)
from toolz import reduce
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots, COLORMAPS
from collections import Counter
from matplotlib.lines import Line2D
from aging.organization.dataframes import load_male_long_df, load_female_long_df, DF_PATHS
from tqdm import tqdm

In [ ]:
def merge(left, right):
    return pd.merge(left, right, left_index=True, right_index=True, how='outer')

In [ ]:
format_plots()

In [ ]:
cmm = COLORMAPS.ont_male
cmf = COLORMAPS.ont_female
df = load_male_long_df(average_weeks=True, merge_size=False, merge_ages=True, df_path=DF_PATHS.usage_male).groupby(['age','mouse']).mean()
fdf = load_female_long_df(average_weeks=True, merge_size=False, filter_female=True, merge_ages=True, df_path=DF_PATHS.usage_female).groupby(['age','mouse']).mean()

In [ ]:
age = df.index.get_level_values("age")
bins = pd.cut(age, 40, labels=False)
sort_syllables = df.groupby(bins, observed=True).mean().quantile(0.95).sort_values(ascending=False).index

## Try looking at variance

In [ ]:
windows = np.arange(1, 51)

n_sylls = 25
roll_size = 7

norms = {}
for mouse, _df in df.groupby('mouse'):
    m_age = _df.index.get_level_values('age')
    cuts = pd.cut(m_age, windows)

    samples = []
    index = []
    for _bin, win_df in _df.groupby(cuts, observed=True):
        _sample = win_df.iloc[:2].mean()
        samples.append(_sample)
        index.append(win_df.index[0])
    samples = pd.DataFrame(samples, index=pd.MultiIndex.from_tuples(index, names=win_df.index.names))[sort_syllables[:n_sylls]]

    smooth_samples = samples.rolling(roll_size, center=True).std().mean(1)
    _ages = smooth_samples.index.get_level_values('age').astype(int)
    norms[mouse] = pd.Series(smooth_samples.to_numpy(), index=_ages, name=mouse)
    norms[mouse] = norms[mouse][~norms[mouse].index.duplicated(keep='first')]

norms = reduce(merge, norms.values())

mask = norms.isna().sum(1) < 10
norms = norms[mask]
norms_melt = norms.reset_index().melt(id_vars='age', var_name='mouse', value_name='ar')
# Convert 'mouse' to category or string
#norms_melt['mouse'] = norms_melt['mouse'].astype('category')

fnorms = {}
for mouse, _df in fdf.groupby('mouse'):
    m_age = _df.index.get_level_values('age')
    cuts = pd.cut(m_age, windows)

    samples = []
    index = []
    for _bin, win_df in _df.groupby(cuts, observed=True):
        _sample = win_df.iloc[:2].mean()
        samples.append(_sample)
        index.append(win_df.index[0])
    samples = pd.DataFrame(samples, index=pd.MultiIndex.from_tuples(index, names=win_df.index.names))[sort_syllables[:n_sylls]]

    smooth_samples = samples.rolling(roll_size, center=True).std().mean(1)
    _ages = smooth_samples.index.get_level_values('age').astype(int)
    fnorms[mouse] = pd.Series(smooth_samples.to_numpy(), index=_ages, name=mouse)
    fnorms[mouse] = fnorms[mouse][~fnorms[mouse].index.duplicated(keep='first')]

fnorms = reduce(merge, fnorms.values())

mask = fnorms.isna().sum(1) < 10
fnorms = fnorms[mask]
fnorms_melt = fnorms.reset_index().melt(id_vars='age', var_name='mouse', value_name='ar')
#fnorms_melt['mouse'] = fnorms_melt['mouse'].astype('category')

In [ ]:
fig = figure(1.3, 1.3)
ax = sns.lineplot(data=norms_melt, x='age', y='ar', errorbar='sd', err_kws=dict(lw=0), color=COLORMAPS.ont_male(255), label="M")
sns.lineplot(data=fnorms_melt, x='age', y='ar', errorbar='sd', ax=ax, err_kws=dict(lw=0), color=COLORMAPS.ont_female(255), label="F")
ax.set(ylabel="Variance (sd)", xlabel="Age (weeks)")
legend(ax)
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig3"/ 'aging_rate_longv2.pdf')

In [ ]:
## stats

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.tools import pinv_extended  
from statsmodels.stats.anova import anova_lm
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tabulate import tabulate

In [ ]:
import statsmodels.formula.api as smf

def ow_anova(data, dependent_var, factor1):
    """
    Perform two-way ANOVA.
    
    Parameters:
    data (pd.DataFrame): The input data frame.
    dependent_var (str): The name of the dependent variable column.
    factor1 (str): The name of the first factor column.
    factor2 (str): The name of the second factor column.
    
    Returns:
    pd.DataFrame: The ANOVA results.
    """
    
    # Align the data
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    # Fit an OLS model on the ranked data
    model = ols(f'ranked_response ~ {factor1}', data=align_data).fit()
    # Perform ANOVA on the fitted model
    anova_results = sm.stats.anova_lm(model, typ=3)
    
    # Convert results to a DataFrame
    anova_df = pd.DataFrame(anova_results)
    
    return anova_df



def fit_1mixed_linear_models(data, dependent_var, factor1, group):
    
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]

    #formula = f'ranked_response ~ {factor1}*{group}'
    formula = f'ranked_response ~ {factor1}'
        
    # Fit the mixed linear model
    model = smf.mixedlm(formula, align_data, groups=f'{group}')
    result = model.fit()
    return result


def fit_2mixed_linear_models(data, dependent_var, factor1,factor2, group):
    
    align_data = data.copy()
    align_data['ranked_response'] = align_data[[dependent_var]]
    formula = f'ranked_response ~ {factor1}*C({factor2})'
        
    # Fit the mixed linear model
    model = smf.mixedlm(formula, align_data, groups=f'{group}')
    result = model.fit()
    return result


def extract_summary_to_dataframe(results, num_tests):
    summary_data = []

    for var, result in results.items():
        summary = result.summary()
        coefs = result.params
        pvalues = result.pvalues
        conf_int = result.conf_int()
        
        for param in coefs.index:
            pvalue_corrected = min(pvalues[param] * num_tests, 1.0)  # Apply Bonferroni correction
            
            summary_data.append({
                'Dependent Variable': var,
                'Parameter': param,
                'Coefficient': coefs[param],
                'P-value': pvalues[param],
                'Corrected P-value': pvalue_corrected,
                'CI Lower': conf_int[0][param],
                'CI Upper': conf_int[1][param]
            })
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

In [ ]:
mdf = norms_melt.copy()
fdf = fnorms_melt.copy()

fdf['sex']=str('f')
mdf['sex']=str('m')

data = pd.concat([mdf, fdf], ignore_index=True)

In [ ]:
results={}

syll = ['ar']
temp = data.dropna().copy()
temp.reset_index(inplace=True)

for i in syll:
    result=fit_2mixed_linear_models(temp, i, 'age','sex','mouse')
    results[i] = result
num_tests = len(data.columns)  # Number of tests is the number of dependent variables (excluding 'age' and 'mouse')
summary_df = extract_summary_to_dataframe(results, num_tests)

# Print the DataFrame nicely
print(summary_df)
summary_df.to_csv('aging_rate_fvm_longv2.csv')